<h3> Imports 

In [2]:
import acoular as ac
from acoupipe.sampler import MicGeomSampler
from acoupipe.datasets.synthetic import DatasetSyntheticConfig
from acoupipe.datasets.synthetic import DatasetSynthetic
from acoupipe.datasets.features import create_feature

from traits.api import Callable, Int, cached_property, Property, Instance

import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.spatial.distance import pdist

/opt/homebrew/Caskroom/miniconda/base/envs/acoupipe_customFeatures/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-03 17:18:13,637	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/homebrew/Caskroom/miniconda/base/envs/acoupipe_customFeatures/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


<h3> Class RandomMicGeomSampler

In [3]:
class RandomMicGeomSampler(MicGeomSampler):

    #: the microphone geometry instance (type :class:`acoular.MicGeom`)
    target = Instance(ac.MicGeom, args=(), desc='microphone geometry whose positions are sampled')

    #: function to initialize microphone positions
    mpos_fn = Callable()

    min_num_mics = Int(1, help="minimum number of microphones required")

    max_num_mics = Int(
        10, help="maximum number of microphones required"
    )

    generator = Instance(np.random.Generator, args=(), desc='random number generator')

    @cached_property # we cache this so that it is only called once
    def _get_mpos_init(self):
        return self.mpos_fn(self.min_num_mics, self.max_num_mics, self.generator)

<h3> Class Config

In [4]:
class testConfig(DatasetSyntheticConfig):
        
    mpos_fn = Callable()
        
    generator = Instance(np.random.Generator, args=(), desc='random number generator')

    def create_micgeom_sampler(self):
        return RandomMicGeomSampler(
            random_var=norm(loc=0, scale=0),
            ddir=np.array([[1.0], [1.0], [1.0]]),
            target=self.noisy_mics,
            min_num_mics=5,
            max_num_mics=5,           
            mpos_fn=self.mpos_fn,
            generator = self.generator,
        )
    
    def _get_mdim(self):
        return None

    #microphone positions feature
    def _get_default_feature_cartesian_coordinates(self, **kwargs):
        def feat_func(sampler):
            pos = sampler[1].target.pos
            return {'cartesian_coordinates': pos}
        return create_feature(
            feat_func, name='cartesian_coordinates', dtype=float, shape=(3, None)
        )
    
    
    #Microphone Positional Encodings from Paper (FM and PM)
    def _get_default_feature_PM_encodings(self, **kwargs):
        def feat_func(sampler):
            alpha = 7
            beta = 4
            pos = sampler[1].target.pos  # (3, N)
            pe = []
            for i in range(0, pos.shape[1]):  # for each microphone
                pe.append(alpha * pos[0][i] * np.cos(2 * np.pi * beta + pos[2][i]))
                pe.append(alpha * pos[0][i] * np.sin(2 * np.pi * beta + pos[2][i]))
                pe.append(alpha * pos[0][i] * np.cos(2 * np.pi * beta + pos[1][i]))
                pe.append(alpha * pos[0][i] * np.sin(2 * np.pi * beta + pos[1][i]))
            pe = np.array(pe)  # (4 * Array Size)
            return {'PM_encodings': pe}
        return create_feature(
            feat_func, name='PM_encodings', dtype=float, shape=(None, None)
        )

    def _get_default_feature_FM_encodings(self, **kwargs):
        def feat_func(sampler):
            alpha = 7
            beta = 4
            pos = sampler[1].target.pos  # (3, N)
            pe = []
            for i in range(0, pos.shape[1]):  # for each microphone
                pe.append(alpha * pos[0][i] * np.cos(pos[2][i]*beta))
                pe.append(alpha * pos[0][i] * np.sin(pos[2][i]*beta))
                pe.append(alpha * pos[0][i] * np.cos(pos[1][i]*beta))
                pe.append(alpha * pos[0][i] * np.sin(pos[1][i]*beta))
            pe = np.array(pe)  # (4 * Array Size) => 4 features per microphone
            return {'FM_encodings': pe}
        return create_feature(
            feat_func, name='FM_encodings', dtype=float, shape=(None, None)
        )

   
        
    



<h3> Main Testing Loop

In [5]:
def random_positions(min_num_mics, max_num_mics, generator):
        
        n = generator.integers(min_num_mics, max_num_mics + 1)   
        
        r = 0.5 * np.sqrt(generator.random(n))
        theta = 2 * np.pi * generator.random(n)

        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = np.zeros(n)

        positions = np.column_stack((x, y, z))
        #Zentrierung
        positions -= positions.mean(axis=0)
        #Skalieren / Normieren der Apertur
        positions /= np.max(pdist(positions))

        return positions.T

test_generator = np.random.default_rng(seed=20006)

config = testConfig(mpos_fn=random_positions, mode="analytic", mic_sig_noise=False, generator=test_generator, min_nsources=1, max_nsources=4)
d1 = DatasetSynthetic(config=config)

#save to .h5 file
d1.save_h5(features=["csm", "cartesian_coordinates", "eigmode", "loc", 'source_strength_analytic' ], f=1000, num=0, split="training", size=10, name="test.h5")
 

/opt/homebrew/Caskroom/miniconda/base/envs/acoupipe_customFeatures/lib/python3.12/site-packages/acoupipe/datasets/features.py:225: Warning: Queried frequency (1000 Hz) not in set of discrete FFT sample frequencies. Using frequency 1071.88 Hz instead.
  fidx = [get_frequency_index_range(self.freq_data.fftfreq(), f_, self.num) for f_ in f]
100%|██████████| 10/10 [00:00<00:00, 163.32it/s]


In [ ]:
#On the fly mit Generator denke ich

#dataset_generator = dataset.generate(
#    features=["loc","source_strength_analytic"], # choose the features to extract
#    f=[1000], # choose the frequencies to extract
#    split='training', # choose the split of the dataset
#    size=1, # choose the size of the dataset
#    )


#for i, data in enumerate(dataset_generator):
        #print(data["idx"], "\n")
        
        #array = dataset.config.micgeom_sampler.target
        #print(array.pos_total.shape)
        #print(array.center)
        #print(array.aperture)